# **Part 2: Setting up `Cart-Pole` Agent.**


- **`Name`** : **Pavaris Asawakijtananont**

- **`Number`** : **65340500037**

## **Base Class**

##### **q**
- calling the action value with using linear approximator to use with Linear Q Learning 

```python
    def q(self, obs, a=None):
        """Returns the linearly-estimated Q-value for a given state and action."""
        obs_val = obs['policy'][0].detach().cpu().numpy()
        if a==None:
            # Get q values from all action in state
            return np.dot(obs_val, self.w)
        else:
            # Get q values given action & state
            return np.dot(obs_val, self.w[:, a])
        # ====================================== #
```

##### **Scale Action**
```python
    def scale_action(self, action):
        return torch.tensor([[action * ((self.action_range[1] - self.action_range[0]) / (self.num_of_action-1 )) + self.action_range[0]]])
```

##### **Select Action**
```python
    def select_action(self, state):
        """ Select an action based on an epsilon-greedy policy. """
        if np.random.rand() < self.epsilon:
            return np.random.randint(0, self.num_of_action)
        else:
            # Exploitation: choose the action with the highest estimated Q-value
            return np.argmax(self.q(state))
```

## **Linear Q Learning**
